# Second Order IIR Cyclic Reduction (CR): 
$y_n + a_1y_{n-1} + a_2y_{n-2} = x_n + b_1x_{n-1} + b_2x_{n-2}$
- block cyclic reduction
    - main function
        - cyclic_reduction_odd
        - cyclic_reduction_even
        - forward
    - support function
        - gaussian_elimination_parameter
        - block filtering factor
- cross_core_cyclic_reduction

In [187]:
import numpy as np
import math 
from iir_function import permute,depermute,prepare_X 
from iir_ph import C_rd,block_homogeneous_solution

np.set_printoptions(precision=6, suppress=True)

In [188]:
# system parameters
a1 = 0.6
a2 = 0.8
b1 = 0.7
b2 = 0.5

xi1 = -5
xi2 = -3
yi1 = 3
yi2 = 7

In [189]:
L = 16
N = 16*L
x = np.arange(N*L)

X = []

for n in range(N):
    X.append(x[n*L:(n+1)*L])

In [190]:
y = np.zeros(N*L)

for n in range(N*L):
    if n == 0:
        y[0] = x[0] + b1*xi1 + b2*xi2 - a1*yi1 - a2* yi2
    elif n == 1:
        y[1] = x[1] + b1*x[0] + b2*xi1 - a1*y[0] - a2*yi1
    else:
        y[n] = x[n] + b1*x[n-1] + b2*x[n-2] - a1*y[n-1] - a2*y[n-2]

In [191]:
X_bar = permute(X)
B_X_bar = prepare_X(X_bar,b2,b1,xi2,xi1)

In [192]:
def gaussian_elimination_parameter(N,a2,a1):
    """Compute gaussian elimination parameters when reducing systems cyclically.

    Parameters
    ----------
    N  : int
        The number of blocks. N is a power of 2.
        
    a2 : float
        The coefficient for input initial state y_{n-2}.
        
    a1 : float
        The coefficient for input initial state y_{n-1}.

    Returns
    -------
    f,e,... : N x 1 ndarray
        Gaussian elimination parameters, where
        d,c,f,e resides in LHS (filtering matrix for Y, i.e., A)
        h,g,p,q resides in RHS (filtering matrix for initial state, yi1 and yi2)
    """
    
    rounds = int(np.log2(N)) + 1
    f = np.zeros(rounds,dtype=float)
    e = np.zeros(rounds,dtype=float)
    d = np.zeros(rounds,dtype=float)
    c = np.zeros(rounds,dtype=float)
    h = np.zeros(rounds,dtype=float) 
    g = np.zeros(rounds,dtype=float)
    p = np.zeros(rounds,dtype=float) 
    q = np.zeros(rounds,dtype=float)
    
    f[0] = a2
    e[0] = a1
    h[0] = a2
    g[0] = a1
    p[1] = a2
    q[1] = a1
    
    for n in range(1,rounds):
        
        d[n] = -f[n-1]**2/e[n-1]
        c[n] = e[n-1] - f[n-1]/e[n-1]
        f[n] = -e[n-1]*d[n]
        e[n] = f[n-1] - e[n-1]*c[n]
        
        h[n] = -e[n-1]*h[n-1]
        g[n] = f[n-1] - e[n-1]*g[n-1]
        
        if n > 1:
            q[n] = -e[n-1]*q[n-1]
            p[n] = f[n-1] - e[n-1]*p[n-1]

    return f,e,d,c,h,g,p,q

In [193]:
class block_cyclic_reduction:
    """Compute multiple output blocks by cyclically reducing system size.
    
    Parameters
    ----------
    N  : int
        The number of blocks. N must be a power of 2 in cyclic reduction.

    L  : int
        The block size. L is a power of 2, which caters to the SIMD length.
    
    a2 : float
        The coefficient for input initial state y_{n-2}.
        
    a1 : float
        The coefficient for input initial state y_{n-1}.

    yi2: float
        The second output initial state.
    
    yi1: float
        The first output initial state.

    W_bar : list of ndarray
        A list of input permuted blocks.

    Returns
    -------
    f,e,... : N x 1 ndarray
        Gaussian elimination parameters, where
        d,c,f,e resides in LHS (filtering matrix for Y, i.e., A)
        h,g,p,q resides in RHS (filtering matrix for initial state, yi1 and yi2)
        
    H_inv : ndarray
        The matrix for convoluting with the last input blocks in block filtering.

    h_yi2 : L x 1 ndarray
        The vector for filtering the second output initial state, i.e., yi2, for computing the last block.

    h_yi1 : L x 1 ndarray
        The vector for filtering the first output initial state, i.e., yi1, for computing the last block.
        
    Y_bar : list of ndarray
        A list of completed output permuted blocks.
    """

    def __init__(self,N,L,a2,a1,yi2,yi1):

        assert math.log2(N).is_integer(), "N must be a power of 2"

        self.a2 = a2
        self.a1 = a1
        self.yi2 = yi2
        self.yi1 = yi1
        
        self.f,self.e,self.d,self.c,self.h,self.g,self.p,self.q = gaussian_elimination_parameter(N,a2,a1)
        self.H_inv,self.h_yi2_odd,self.h_yi1_odd,self.h_yi2_even,self.h_yi1_even = self.block_filtering_parameter(N,L)
        

    def block_filtering_parameter(self,N,L):
        """Compute block filtering matrix and vectors for input blocks and initial states. 
    
        Parameters
        ----------
        N  : int
            The number of blocks. N is a power of 2.
        
        L  : int
            The block size. L is a power of 2, which caters to the SIMD length.
    
        Returns
        -------
        H_inv : ndarray
            The matrix for convoluting with the last input blocks in block filtering.

        h_yi2_odd : L x 1 ndarray
            The vector for filtering yi2, in compututation of y_{N-1}, 
    
        h_yi1_odd : L x 1 ndarray
            The vector for filtering yi1, in compututation of y_{N-1}, 

        h_yi2_even : L x 1 ndarray
            The vector for filtering yi2, in compututation of y_{N-2}, 
    
        h_yi1_even : L x 1 ndarray
            The vector for filtering yi1, in compututation of y_{N-2}, 
        """
        
        rounds = int(np.log2(N))
        h_inv = np.zeros(L,dtype=float)
        
        H_inv = []
        
        for l in range(L):
            
            if l == 0:
                h_inv[l] = 1
            elif l == 1:
                h_inv[l] = -self.e[rounds]
            else:
                h_inv[l] = -self.e[rounds]*h_inv[l-1] - self.f[rounds]*h_inv[l-2]
                
        for l in range(L):
            
            if l == 0:
                H_inv.append(h_inv)
            else:
                H_inv.append(np.concatenate((np.zeros(l),h_inv[:-l])))
                
        h_yi2_odd = self.h[rounds]*h_inv
        h_yi1_odd = self.f[rounds]*np.concatenate(([0],h_inv[:-1])) + self.g[rounds]*h_inv

        h_yi2_even = self.f[rounds]*np.concatenate(([0],h_inv[:-1])) + self.p[rounds]*h_inv
        h_yi1_even = self.q[rounds]*h_inv
        
        return (H_inv,h_yi2_odd,h_yi1_odd,h_yi2_even,h_yi1_even)


    def cyclic_reduction_odd(self,X_bar,ro=0,ind=1):
        """Compute input blocks after cyclic reduction for odd-numbered (bottom side) blocks.
    
        Parameters
        ----------
        X  : list of ndarray
            A list of input blocks.
        
        ro  : int
            The current round of cyclic reduction.

        ind : int
            Either 1 or 2. 1: cyclic reduction for odd-numbered blocks. 2. CR for even-numbered blocks.
    
        Returns
        -------
        X : list of ndarray
            A list of input blocks after gaussian elimination in CR.

        x_last : ndarray
            The last block without appending the initial states. It depends on the value of ind.
        """
        
        N = len(X_bar)
        rounds = int(np.log2(N))
        
        K = 2**(ro+1) # seperation factor
        
        if ro <= rounds:
            
            # top side
            for n in range(N//K):   
                if n == 0:
                    X_bar[K*n+K//2-ind] -= self.f[ro]/self.e[ro]*np.concatenate(([0],X_bar[-ind][:-1])) 
                else:
                    X_bar[K*n+K//2-ind] -= self.f[ro]/self.e[ro]*X_bar[K*n-ind]
                    
            # bottom side
            for n in range(N//K):
                X_bar[K*n+K-ind] -= self.e[ro]*X_bar[K*n+K//2-ind]
            
            ro += 1
            return self.cyclic_reduction_odd(X_bar,ro,ind)
            
        # last round, get the last block without appending the initial states
        else:
            
            L = len(self.H_inv)
            x_last = np.zeros(L)

            # block filtering
            for l in range(L):
                x_last += self.H_inv[l]*X_bar[-ind][l]
                
            return X_bar,x_last

    
    def cyclic_reduction_even(self,X_bar,ro=0):
        """Compute input blocks after cyclic reduction for even-numbered (top side) blocks.
    
        Parameters
        ----------
        X  : list of ndarray
            A list of input blocks.
        
        ro  : int
            The current round of cyclic reduction.
    
        Returns
        -------
        The function of cyclic_reduction_even
            Cyclically compute last even-numbered block, which is at the bottom side.
        """
        
        N = len(X_bar)
        rounds = int(np.log2(N))
        
        K = 2**(ro+1)
                    
        # bottom side
        for n in range(N//K):

            X_bar[K*n+K-1] -= self.f[ro]/self.e[ro]*X_bar[K*n+K//2-1]
            
        # top side
        for n in range(N//K):  
            
            if n == 0:
                X_bar[K*n+K//2-1] -= self.e[ro]*np.concatenate(([0],X_bar[-1][:-1])) 
            else:
                X_bar[K*n+K//2-1] -= self.e[ro]*X_bar[K*n-1]
                
        ro += 1
        # cyclically compute last even-numbered block, which is at the bottom side.
        return self.cyclic_reduction_odd(X_bar,ro,2)
        

    def forward(self,X_bar,x_last_odd,yi2,yi1):
        """Compute all output blocks by filtering with initial odd-numbered blocks. 
    
        Parameters
        ----------    
        X_bar : list of ndarray
            A list of ndarray of input blocks after gaussian elimination.
            
        x_last_odd : ndarray
            The last block, i.e., x_{N-1}, after gaussian elimination without appending the initial states.
            
        yi2 : float
            The second output initial state.
            
        yi1 : float
            The first output initial state.
    
        Returns
        -------
        Y_bar : list of ndarray
            The completed output blocks.
        """
        # forward the rest blocks based on odd-numbered blocks.
        
        N = len(X_bar)
        L = len(X_bar[0])

        Y_bar = np.zeros_like(X_bar)

        rounds = int(np.log2(N))
        
        # get the last block
        Y_bar[-1] = x_last_odd - self.h_yi2_odd*yi2 - self.h_yi1_odd*yi1
        
        # compute the rest blocks        
        for ro in range(rounds-1,-1,-1):
            
            K = 2**(ro+1)
            P = N//K
            
            for n in range(P):
                
                # the first round
                if ro == rounds-1:
                    
                    # the block before last, y_n = x_n - [h c ..] * [yi2 y_{N-1} ..] - [g d ..] * [yi1 yi1 y_{N-1} ..] 
                    yvi2 = np.concatenate(([yi2],Y_bar[-1][:-1]))
                    yvi1 = np.concatenate(([yi1,yi1],Y_bar[-1][:-2])) 
                    h_yi2 = np.concatenate(([self.h[ro]],np.ones(L-1)*self.c[ro+1]))
                    h_yi1 = np.concatenate(([self.g[ro]],np.ones(L-1)*self.d[ro+1]))
                    
                # the rest round   
                else:
                    # the first block, y_n = x_n - dR*y_{N-2} - cR*y_{N-1} 
                    if n == 0:
                        yvi2 = np.concatenate(([yi2],Y_bar[N-K-1][:-1]))
                        yvi1 = np.concatenate(([yi1],Y_bar[-1][:-1])) 
                        h_yi2 = np.concatenate(([self.h[ro]],np.ones(L-1)*self.d[ro+1]))
                        h_yi1 = np.concatenate(([self.g[ro]],np.ones(L-1)*self.c[ro+1]))

                    # the second block, y_n = x_n - dR*y_{N-1} - c*y_0 
                    # the rest block, y_n = x_n - d*y_{n-2} - c*y_{n-1} 
                    else: 
                        yvi2 = np.concatenate(([yi1],Y_bar[-1][:-1])) if n == 1 else Y_bar[K*(n-1)-1]
                        yvi1 = Y_bar[K*n-1]
                        h_yi2 = self.d[ro+1]
                        h_yi1 = self.c[ro+1]

                Y_bar[K*n+K//2-1] = X_bar[K*n+K//2-1] - h_yi2*yvi2 - h_yi1*yvi1      
        return Y_bar

    def forward_even(self,X_bar,x_last_even,yi2,yi1):
        """Compute all output blocks by filtering with initial even-numbered blocks. 
    
        Parameters
        ----------    
        X_bar : list of ndarray
            A list of ndarray of input blocks after gaussian elimination.
            
        x_last_even : ndarray
            The second last block, i.e., x_{N-2}, after gaussian elimination without appending the initial states. 
            
        yi2 : float
            The second output initial state.
            
        yi1 : float
            The first output initial state.

        Returns
        -------
        Y_bar : list of ndarray
            The completed output blocks.
        """
        
        N = len(X_bar)
        L = len(X_bar[0])

        Y_bar = np.zeros_like(X_bar)

        rounds = int(np.log2(N))
        
        # get the last block
        Y_bar[-2] = x_last_even - self.h_yi2_even*yi2 - self.h_yi1_even*yi1
        
        # compute the rest blocks until the second last round     
        for ro in range(rounds-1,0,-1):
            
            K = 2**(ro+1)
            P = N//K
            
            for n in range(P):
                
                # the first round
                if ro == rounds-1:
                    
                    # the block before last, y_n = x_n - [h c ..] * [yi2 y_{N-1} ..] - [g d ..] * [yi1 yi1 y_{N-1} ..] 
                    yvi2 = np.concatenate(([yi2,yi2],Y_bar[-2][:-2]))
                    yvi1 = np.concatenate(([yi1],Y_bar[-2][:-1])) 
                    h_yi2 = np.concatenate(([self.p[ro]],np.ones(L-1)*self.d[ro+1]))
                    h_yi1 = np.concatenate(([self.q[ro]],np.ones(L-1)*self.c[ro+1]))
                    
                # the rest round   
                else:
                    # the first block, y_n = x_n - dR*y_{N-2} - cR*y_{N-1} 
                    if n == 0:
                        
                        yvi2 = np.concatenate(([yi2],Y_bar[N-K-2][:-1]))
                        yvi1 = np.concatenate(([yi1],Y_bar[-2][:-1])) 
                        h_yi2 = np.concatenate(([self.p[ro]],np.ones(L-1)*self.d[ro+1]))
                        h_yi1 = np.concatenate(([self.q[ro]],np.ones(L-1)*self.c[ro+1]))

                    else: 
                        # the second block, y_n = x_n - dR*y_{N-1} - c*y_0 
                        if n == 1:
                            yvi2 = np.concatenate(([yi2],Y_bar[-2][:-1]))
                            
                        # the rest block, y_n = x_n - d*y_{n-2} - c*y_{n-1} 
                        else:
                            yvi2 = Y_bar[K*(n-1)-2]

                        yvi1 = Y_bar[K*n-2] 
                        h_yi2 = self.d[ro+1]
                        h_yi1 = self.c[ro+1]

                Y_bar[K*n+K//2-2] = X_bar[K*n+K//2-2] - h_yi2*yvi2 - h_yi1*yvi1

        # compute odd-numbered block
        for n in range(N//2):
    
            if n == 0:
                yvi2 = np.concatenate(([yi2],Y_bar[-2][:-1]))
            else:
                yvi2 = Y_bar[2*(n-1)]
    
            yvi1 = Y_bar[2*n] 
            h_yi2 = self.d[1]
            h_yi1 = self.c[1] 
    
            Y_bar[2*n+1] = X_bar[2*n+1] - h_yi2*yvi2 - h_yi1*yvi1

        return Y_bar

    def compute(self,X_bar):

        # # test for even-numbered blocks
        # X_bar,x_last = self.cyclic_reduction_even(X_bar)
        # Y_bar = self.forward_even(X_bar,x_last,self.yi2,self.yi1)

        # compute odd-numbered blocks first as default.
        X_bar,x_last = self.cyclic_reduction_odd(X_bar)
        Y_bar = self.forward(X_bar,x_last,self.yi2,self.yi1)
              
        return Y_bar
    

In [194]:
BCR = block_cyclic_reduction(N,L,a2,a1,yi2,yi1)
Y_bar = BCR.compute(B_X_bar)

Y = depermute(Y_bar)

assert np.isclose(np.ravel(Y),y).all()
print('ok')

ok


### Cross-Core Computation

In [196]:
class block_cross_core_cyclic_reduction(block_cyclic_reduction,block_homogeneous_solution):
    """Compute multiple output blocks in multiple cores simultaneously by cyclic reduction.
    
    Parameters
    ----------
    N  : int
        The number of blocks. N is a multiple of L.

    L  : int
        The block size. L is a power of 2, which caters to the SIMD length.
    
    a2 : float
        The coefficient for input initial state y_{n-2}.
        
    a1 : float
        The coefficient for input initial state y_{n-1}.

    yi2: float
        The second output initial state.
    
    yi1: float
        The first output initial state.

    Returns
    -------
    C_cross : 2 x 2 ndarray
        Block recursive parameter for cross core computation.

    C_cross_dict : dictionary of ndarray
        A dictionary of the block recursive doubling multiplier for multi-core processing, which has the following directory:
                                   key                                           value
        first layer:   l (the number of recursive round)               second layer dictionary.
        second layer: '-2' or '-1'(C_cross matrix for xv2 or xv1)         third layer ndarray
        third layer:   0 or 1 (elements in C added to xv2 or xv1)             L x 1 ndarray

    Y_bar_list : list of list of ndarray
        A list of "multiple completed output blocks" computed by each core.
    """   
    
    def __init__(self,N,L,a2,a1,yi2,yi1):
        block_cyclic_reduction.__init__(self,N,L,a2,a1,yi2,yi1)
        block_homogeneous_solution.__init__(self,N,L,a2,a1,yi2,yi1)

        # C_cross can be computed by either from ph or cr.
        C_cross = C_rd(L,self.h2,self.h1,c_cross=True)
        self.C_dict_cross = C_rd(L=L,C=C_cross)
    
    def compute(self,X_bar_list):

        L = len(X_bar_list)

        xv2_cross = np.zeros(L)
        xv1_cross = np.zeros(L)
        
        x_last_odd_list = []
        x_last_even_list = []
        X_bar_tilde_list = []

        # step 1: compute the last two blocks by divide and conquer (without adding initials) in each core
        for l in range(L):

            # Note: this is not exactly what we want for passing two copies of inputs 
            # to two cyclic reduction functions in two directions.
            # cyclic reduction in two directions should be inherently not interfering each other,
            # which is good for C++ since we can directly working on the same data set (X_bar) without copying it.
            # but now this is not the case because the even-numbered blocks computation requires raw odd-numbered blocks,
            # while odd-numbered blocks requires raw even-numbered blocks.
            X_bar_tilde,x_last_odd = super().cyclic_reduction_odd(X_bar_list[l].copy()) 
            _,x_last_even = super().cyclic_reduction_even(X_bar_list[l].copy())
            
            xv2_cross[l] = x_last_even[-1]
            xv1_cross[l] = x_last_odd[-1]
            
            x_last_even_list.append(x_last_even)
            x_last_odd_list.append(x_last_odd)
            X_bar_tilde_list.append(X_bar_tilde)

        # step 2: cross-core recursive doubling (with initial appending) to get the initial states for all cores.
        yv2_cross,yv1_cross = super().block_recursive_doubling(self.C_dict_cross,xv2_cross,xv1_cross,self.yi2,self.yi1)

        Y_bar_list = []

        # step 3: inner-core cyclic reduction to get all output blocks
        for l in range(L):

            if l == 0:
                yi2 = self.yi2
                yi1 = self.yi1
            else:
                yi2 = yv2_cross[l-1]
                yi1 = yv1_cross[l-1]

            Y_bar_list.append(super().forward(X_bar_tilde_list[l],x_last_odd_list[l],yi2,yi1))

        return Y_bar_list

In [197]:
L = 8
N = 4*L
pa = L
x = np.arange(N*L*pa)*8

X = []

for n in range(N*pa):
    X.append(x[n*L:(n+1)*L])

In [198]:
y = np.zeros(N*L*pa)

for n in range(pa*N*L):
    if n == 0:
        y[0] = x[0] + b1*xi1 + b2*xi2 - a1*yi1 - a2* yi2
    elif n == 1:
        y[1] = x[1] + b1*x[0] + b2*xi1 - a1*y[0] - a2*yi1
    else:
        y[n] = x[n] + b1*x[n-1] + b2*x[n-2] - a1*y[n-1] - a2*y[n-2]

In [199]:
W_list = []
B_X_bar_list = []

for p in range(pa):
    X_bar = permute(X[p*N:(p+1)*N])
    if p == 0:
        B_X_bar = prepare_X(X_bar,b2,b1,xi2,xi1)
    else:
        B_X_bar = prepare_X(X_bar,b2,b1,x[p*N*L-2],x[p*N*L-1])
        
    B_X_bar_list.append(B_X_bar)

BCCCR = block_cross_core_cyclic_reduction(N,L,a2,a1,yi2,yi1)
Y_bar_list = BCCCR.compute(B_X_bar_list)

Y = []

for p in range(pa):
    Y.append(depermute(Y_bar_list[p]))
assert np.isclose(np.ravel(Y),y).all()
print('ok')

ok
